# Facts and variables

When connecting to an host, ansible creates a dictionary with a huge set of useful informations.

This step is named **fact gathering**. 

You can inspect facts using the `setup` module.

In [1]:
cd /notebooks/exercise-03

/notebooks/exercise-03


In [2]:
# Let's run the setup module

!ansible -i inventory -m setup localhost 

 [WARNING]: Unable to parse /notebooks/exercise-03/inventory as an inventory
source

 [WARNING]: No inventory was parsed, only implicit localhost is available

 [WARNING]: Could not match supplied host pattern, ignoring: all

 [WARNING]: provided hosts list is empty, only localhost is available

localhost | SUCCESS => {
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "172.17.0.2"
        ], 
        "ansible_all_ipv6_addresses": [], 
        "ansible_apparmor": {
            "status": "disabled"
        }, 
        "ansible_architecture": "x86_64", 
        "ansible_bios_date": "11/28/2019", 
        "ansible_bios_version": "1.3.1", 
        "ansible_cmdline": {
            "BOOT_IMAGE": "(hd0,gpt7)/vmlinuz-5.14.18-100.fc33.x86_64", 
            "quiet": true, 
            "rd.lvm.lv": "fedora_localhost-live/swap", 
            "resume": "/dev/mapper/fedora_localhost--live-swap", 
            "rhgb": true, 
            "ro": true, 
            "root": "/dev/m

#### Exercise

 - run the following command
 - what does it do?
 - where's  its output ?
 - use the json python module to make it readable

In [ ]:
!ansible >/dev/null -i inventory -m setup localhost --tree host_status

In [ ]:
# Solution
!tree host_status
import json
ret = json.load(open('host_status/localhost'))
f = ret['ansible_facts'] # .keys()
f['ansible_all_ipv4_addresses']

#### Exercise

Gather the following information from the output of the previous command.

  - hostname
  - first ip address
  - distribution
  - kernel release



In [ ]:
# use this cell for the exercise

#### Exercise

modify [this playbook](/edit/notebooks/exercise-03/get-facts.yml) to print:


  - hostname
  - first ip address
  - distribution
  - kernel release


In [ ]:
!ansible-playbook get-facts.yml

## Facts vs Variables

Essentially, facts are per-host variables gathered by ansible.

In the vault lesson, we've seen variables in action loaded it via `-e @variables.yml`.

Now we'll define them with 

```yaml
- hosts: localhost
  vars:
  - one_variable: 1
  - another_variable: 
      that_is: a_map
```

## Templates & filters

Ansible can create files using the [jinja2](http://jinja.pocoo.org/docs/2.10/) template engine.

Filters are jinja2 functions to process facts and variables.

### jinja2

Here's a template example

```html
<html>
  <body>

# template.j2
This is a static line while the following one
expands the ansible_hostname variable {{ansible_hostname}} 

Now we process a simple number {{ 3.1415 | int }}

Or take the first letter from a variable: {{ ansible_hostname[0] }}
  </body>
</html>
```

You can process entries inside a template with filters.


In [ ]:
# Use the debug module to process a variable
# Modify the command to print the last two letters.
!ansible localhost -mdebug -a"var=ansible_user[:2]"

In [ ]:
!cat vars-and-facts.yml

In [ ]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags variables

## filters

Inside `{{` braces `}}` you can process expressions using filters.

```

  - debug:
      msg: >
        Floor it  {{ ( one + pi ) | int }}


```

A simple filter using `debug` in a playbook.

```
- name: We already found a simple filter mapping entries to int
  debug: msg="{{ 3.1415 | int }}"
```

A filter is essentially a function returning a function, like a lambda.

```
int_filter = lambda x: int(x)
```

A more complex filter: 
```
- name: This is a getter
  debug: 
    msg: >
      {{ ['host1', 'host2'] | 
            map('extract', hostvars, ['key1', .. , 'keyN']) 
      }}
```

where
```
hostvars_getter = lambda host: hostvars[host]['key1'][..]['keyN']
```

You can pipeline filters and test them incrementally.

We'll see further examples when processing `hostvars`

In [ ]:
# This playbook mixes variables and facts
!ansible-playbook vars-and-facts.yml --tags filters

#### Exercise

Edit [the last section of vars-and-facts.yml](/edit/notebooks/exercise-03/vars-and-facts.yml) so that it shows:
    
    - the free percentage of every device

Hints:
    
    - use the playbook as a reference
    - iterate thru server facts

In [ ]:
!ansible-playbook vars-and-facts.yml --tags exercise